In [ ]:
import cv2
import numpy as np
import os
from IPython.display import HTML, display
import base64

print("✅ 라이브러리가 준비되었습니다.")

In [ ]:
import firebase_admin
from firebase_admin import credentials, storage
from google.colab import userdata
import json
import os

# cred = credentials.Certificate("./serviceAccountKey.json")

key_json = userdata.get('firebase') #colab 환경

try:
    with open('firebase_key.json', 'w') as f:
        json.dump(json.loads(key_json), f)
    key_path = 'firebase_key.json'
except json.JSONDecodeError:
    # If the key is not valid JSON, assume it's already a file path
    key_path = key_json


cred = credentials.Certificate(key_path)

if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'storageBucket': 'fairplayfairy-3e2eb.firebasestorage.app'
    })
    print("Firebase 앱이 성공적으로 초기화되었습니다.")
else:
    print("Firebase 앱이 이미 초기화되어 있습니다.")

# Storage 버킷 객체 가져오기 예시
bucket = storage.bucket()
print("Storage 버킷에 접근 성공:", bucket.name)

In [ ]:
import cv2
import numpy as np
import traceback

def track_aim_in_video(input_path, output_path=None, warmup_frames=0):
    """
    스킵(3초) 로직 제거. WebM 메타 부정확 시에도 전체 프레임 처리.
    warmup_frames: 앞 부분은 계산만 하고 aim_movements에 미저장.
    """
    try:
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"Error: 영상을 열 수 없습니다. 경로: {input_path}")
            return None

        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv2.CAP_PROP_FPS)
        try:
            total_frames_raw = cap.get(cv2.CAP_PROP_FRAME_COUNT)
            total_frames = int(total_frames_raw)
            if total_frames <= 0:
                total_frames = None
        except:
            total_frames = None

        if width == 0 or height == 0:
            print("Warning: 해상도 정보 실패")
            cap.release()
            return None

        # 첫 프레임 읽기
        ret, prev_frame = cap.read()
        if not ret:
            print("Error: 첫 프레임 읽기 실패")
            cap.release()
            return None

        roi_x, roi_y = int(width * 0.2), int(height * 0.2)
        roi_w, roi_h = int(width * 0.6), int(height * 0.6)
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        prev_roi  = prev_gray[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

        aim_movements = []
        out = None
        if output_path:
            use_fps = fps if 1 <= fps <= 480 else 60
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, int(use_fps), (width, height))

        frame_index = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            roi  = gray[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

            flow = cv2.calcOpticalFlowFarneback(
                prev_roi, roi, None,
                0.5, 3, 15, 3, 5, 1.2, 0
            )
            dx = float(np.mean(flow[...,0]))
            dy = float(np.mean(flow[...,1]))
            aim_dx, aim_dy = -dx, -dy

            if frame_index >= warmup_frames:
                aim_movements.append((aim_dx, aim_dy))

            if out:
                out.write(frame)

            prev_roi = roi
            frame_index += 1

        cap.release()
        if out:
            out.release()
        return aim_movements

    except Exception as e:
        print(f"❌ track_aim_in_video 함수 예외: {e}")
        traceback.print_exc()
        return None

⚠️ 파일 경로를 먼저 확인해주세요.


In [ ]:
import firebase_admin
from firebase_admin import credentials, storage
from google.colab import userdata
import json
import tempfile
import os
from joblib import Parallel, delayed

# --- 1. 단일 영상 처리 함수 재정의 (Firebase 초기화 로직 추가) ---
def process_video_task(blob_name, bucket_name, firebase_key_json, total_count, current_index):
    """
    하나의 비디오 파일을 처리하는 작업 단위.
    각 프로세스 내부에서 Firebase를 초기화합니다.
    """
    progress = f"[{current_index}/{total_count}]"
    
    # --- 각 프로세스별 Firebase 초기화 ---
    try:
        if not firebase_admin._apps:
            # 전달받은 키(JSON)로 인증서 생성 및 초기화
            cred = credentials.Certificate(json.loads(firebase_key_json))
            firebase_admin.initialize_app(cred, {'storageBucket': bucket_name})
        
        # 초기화 후 버킷 및 블롭 객체 가져오기
        bucket = storage.bucket()
        video_blob = bucket.blob(blob_name)

    except Exception as e:
        print(f"{progress} ❌ Firebase 초기화/연결 실패: {blob_name} ({e})")
        return None

    # --- 기존 파일 처리 로직 (동일) ---
    fd, temp_path = tempfile.mkstemp(suffix=".webm")
    os.close(fd)
    try:
        try:
            video_blob.download_to_filename(temp_path)
        except Exception as e:
            print(f"{progress} ❌ 다운로드 실패: {video_blob.name} ({e})")
            return None
        if not os.path.exists(temp_path) or os.path.getsize(temp_path) == 0:
            print(f"{progress} ❌ 빈 파일 → 건너뜀: {video_blob.name}")
            return None
        aim_data = track_aim_in_video(temp_path, output_path=None)
        if aim_data is None:
            print(f"{progress} ⚠️ 분석 실패: {video_blob.name}")
            return None
        print(f"{progress} ✅ 수집: {video_blob.name} ({len(aim_data)} 프레임)")
        return {'name': video_blob.name, 'data': aim_data}
    finally:
        try:
            if os.path.exists(temp_path):
                os.remove(temp_path)
        except Exception as e:
            print(f"{progress} ❗ 임시파일 삭제 실패: {e}")

# --- 2. 메인 로직 (병렬 처리 실행) ---

# 병렬 처리를 시작하기 전에 필요한 정보를 미리 변수에 저장
firebase_key = userdata.get('firebase') # Colab Secrets에서 키 가져오기
bucket_name = bucket.name # 2번 셀에서 초기화된 bucket 객체 사용

videos = list(bucket.list_blobs(prefix='videos/'))
webm_files = [v for v in videos if v.name.endswith('.webm')]

if not webm_files:
    print("⚠️ 처리할 .webm 없음")
else:
    total_files = len(webm_files)
    print(f"총 {total_files}개 영상 병렬 처리 시작 (multiprocessing)")

    # blob 객체 대신, 간단한 문자열 데이터(이름, 버킷, 키)를 전달
    results = Parallel(n_jobs=-1)(
        delayed(process_video_task)(
            blob.name, bucket_name, firebase_key, total_files, i
        ) 
        for i, blob in enumerate(webm_files, start=1)
    )

    all_aim_data = [r for r in results if r is not None]
    print(f"\n--- 완료 ---\n✔️ 성공: {len(all_aim_data)} / {total_files}")

In [ ]:
from google.colab import drive
import pandas as pd
import os

# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

# 2. 영상 처리 결과를 DataFrame으로 변환
# 예시: all_aim_data = [{'name': 파일명, 'data': [(dx, dy), ...]}, ...]
rows = []
for item in all_aim_data:
    name = item['name']
    for idx, (dx, dy) in enumerate(item['data']):
        rows.append({'video': name, 'frame': idx, 'dx': dx, 'dy': dy})

df = pd.DataFrame(rows)

# 3. 엑셀로 저장 
save_path = '/content/drive/MyDrive/aim_data.xlsx'
df.to_excel(save_path, index=False)
print(f"엑셀 파일 저장 완료: {save_path}")